In [1]:
import requests
import json

In [2]:
class Embedder:
    def __init__(self, model_name):
        # Updated base URL to the new endpoint
        base_url = "http://ray.ml.uat.us.workloads.elizacloud.com/serve/"
        
        if model_name == "e5_mistral_embed_384":
            self.url = base_url + "e5embed384"
        else:
            raise ValueError(f"Unsupported model name: {model_name}")

        self.headers = {"Content-Type": "application/json"}
        print(f"[INFO] Initialized Embedder with URL: {self.url}")

    def get_embeddings(self, texts):
        data = {"texts": texts}
        print(f"[DEBUG] Sending data: {data} to {self.url}")
        
        try:
            response = requests.post(self.url, headers=self.headers, json=data, verify=False)
            print(f"[DEBUG] Received response with status code: {response.status_code}")
            response.raise_for_status()

            response_json = response.json()
            print("[INFO] API Response:", response_json)

            # Check for embedding field in the response
            if "dense_embeddings" in response_json:
                embeddings = response_json["dense_embeddings"]
                print("[INFO] Found dense_embeddings directly in response.")
                return embeddings
            elif "result" in response_json:
                # Try to convert the result if it's a string
                if isinstance(response_json["result"], str):
                    print("[DEBUG] Parsing 'result' string into JSON...")
                    response_json["result"] = json.loads(response_json["result"])
                    
                if "dense_embeddings" in response_json["result"]:
                    embeddings = response_json["result"]["dense_embeddings"]
                    print("[INFO] Found dense_embeddings inside result.")
                    return embeddings
                
            raise ValueError(f"Unexpected API response format: {response_json}")
        except requests.exceptions.RequestException as e:
            print(f"[ERROR] API Request Error: {e}")
            return None
        except ValueError as e:
            print(f"[ERROR] Response Parsing Error: {e}")
            return None

In [3]:
e5_embedder = Embedder("e5_mistral_embed_384")

test_sentences = [
    "Hello, world!"
]
print(f"[INFO] Requesting embeddings for sentences: {test_sentences}")

# Get the embeddings
embedding_vectors = e5_embedder.get_embeddings(test_sentences)

if embedding_vectors is not None:
    print("Embedding Vectors:")
    print(json.dumps(embedding_vectors, indent=2))
else:
    print("Unable to retrieve embedding vectors.")

[INFO] Initialized Embedder with URL: http://ray.ml.uat.us.workloads.elizacloud.com/serve/e5embed384
[INFO] Requesting embeddings for sentences: ['Hello, world!']
[DEBUG] Sending data: {'texts': ['Hello, world!']} to http://ray.ml.uat.us.workloads.elizacloud.com/serve/e5embed384
[ERROR] API Request Error: HTTPConnectionPool(host='ray.ml.uat.us.workloads.elizacloud.com', port=80): Max retries exceeded with url: /serve/e5embed384 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x749c16250a60>, 'Connection to ray.ml.uat.us.workloads.elizacloud.com timed out. (connect timeout=None)'))
Unable to retrieve embedding vectors.
